<a href="https://www.kaggle.com/williamkaiser/covid-19-data-analysis?scriptVersionId=89705836" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Analysis of Papers Written About COVID 19 Based on Impact and Text Content
This started off really organized, but now it is not. I need things to run fast!!!! Fuck!

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk
import os

# Monitors the Status of the Project
from tqdm import tqdm
tqdm.pandas()

# The Dark Arts
from warnings import filterwarnings
filterwarnings("ignore")

In [ ]:
# SYSTEM VARIABLES
FRAC_DATASET = 1 # Runs analysis on only one percent of the dataset
INC_THRESH = 0.25 # At least 25 % of the impact data has to be present

data = pd.read_csv("/kaggle/input/CORD-19-research-challenge/metadata.csv", dtype="string")

data.head()

In [ ]:
# Removes the non-essential data
data = data.sample(frac=FRAC_DATASET)

N_ROWS = len(data.index)
N_COLS = len(data.columns)

data.dropna(subset=['doi', 'title', 'abstract', 'url', 'cord_uid'], inplace=True)

print(f"number of columns:       {N_COLS:10d}")
print(f"original number of rows: {N_ROWS:10d}")
print(f"new number of rows:      {len(data.index):10d}")
print(f"Percent Removed:         {(1 - len(data.index) / N_ROWS) * 100:9.2f}%")  

### Exploratory Data Analysis
Here we learn more about the dataset and the papers that it contains. This will be used to guide later analysis.

In [ ]:
from datetime import datetime
import matplotlib.pyplot as plt

def get_date(date):
    if date.find("-") != -1:
        return datetime.strptime(date, "%Y-%m-%d")
    else:
        return datetime.strptime(date, "%Y")

# Time Created Histogram
publish_times = data['publish_time'].dropna().apply(get_date)
publish_times = publish_times[publish_times > datetime.strptime("2000-1-1", "%Y-%m-%d")]
plt.hist(publish_times, bins=20)
plt.title("When Papers on COVID-19 Were Published")
plt.xlabel("Date Published")
plt.ylabel("Number of Papers")
plt.savefig('paper_ages.png')
plt.show()

In [ ]:
# Comparing the length of abstracts to the length of titles to determine compute time
abstract_char_count = data['abstract'].apply(len).sum()
title_char_count = data['title'].apply(len).sum()

print(f"abstracts are {abstract_char_count / title_char_count:10.2f}x larger")

# Impact Data and Analysis
Here, I am trying to learn more about what kind of impact these papers have and their distributions. Pending these results, I will consider splitting my analysis between high and low impact papers. Additionally, I need to linearize this data regardless.

In [ ]:
# Gets a list of API Calls to make (will be joined back up later)
def get_request(doi): 
    return f"https://api.altmetric.com/v1/doi/{doi}"
    
calls = data['doi'].apply(lambda doi: f"https://api.altmetric.com/v1/doi/{doi}").to_frame()
calls['cord_uid'] = data['cord_uid']  
calls.to_csv('altmetric_calls.csv', index=False)
del calls

In [ ]:
INC_THRESH = 0.5
# Reads the impact data generated from alt metric to a file
impact = pd.read_csv('/kaggle/input/impact-data/impact_upload.csv')

# Displays the Percentage of Null Values Per Column
null_values = impact.isna().sum().sort_values()
print("Where the most ")
null_values[null_values < INC_THRESH * len(null_values)]


In [ ]:
# Drops the DOI from impact (only other overlapping column)
impact.drop(['doi'], axis=1, inplace=True)

# Joins the two data frames together
data = data.set_index('cord_uid').join(impact.set_index('cord_uid'))

# Removes NAs for any of the categories in place
data.dropna(subset=['score', 'readers_count', 'posts', 'accounts'], inplace=True) # Gets rid of everything without a full text

##### Good paper impact data:
- score
- readers_count
- posts
- accounts


# Textual Analysis
1. Remove non-essential words (what, is, a)
2. Turn each remaining word into its root (walking -> walk)
3. Unsupervised learning on each title (going to expand this to abstracts in the future, but it took too long for testing)
4. Apply the machine learning model to turn each sentence to a list of 100 numbers 

Note: a shift to the spacy bio parser will have to take place to actually read this information

In [ ]:
# Language detection

# Note: reading another paper that did textual analysis on this, they removed all of the non-english papers.

# Only a very small percent of these papers were non-english, but I thought it would be a good idea
!pip install langdetect
from langdetect import detect
from langdetect import DetectorFactory

# Set the seed
DetectorFactory.seed = 1231123123

print("Starting language detection")

def get_language(row):
    """Gets the languages of each paper using fancy machine-learning"""
    try:
        return detect(' '.join(row[:min(len(row), 0)]))
    except:
        return 'en'
    
data['lang'] = data['abstract'].progress_apply(get_language)

print(f"Length Before {len(data.index):6d}")


data = data[data['lang'] == 'en']
print(f"Length After  {len(data.index):6d}")

In [ ]:
#NLP 
null = !pip install -U spacy[cuda]
null = !pip install scispacy
null = !pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_core_sci_lg-0.4.0.tar.gz
#null = !pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_core_sci_sm-0.4.0.tar.gz

del null # Just mutes the text

import spacy
from spacy.lang.en.stop_words import STOP_WORDS

In [ ]:
data = data.iloc[:80000,:] # Goes down to 60,000 entries (I am super worried about ram wtf!)

In [ ]:
import string

punctuations = string.punctuation
stopwords = list(STOP_WORDS)
stopwords[:10]

In [ ]:
custom_stop_words = [
    'doi', 'preprint', 'copyright', 'peer', 'reviewed', 'org', 'https', 'et', 'al', 'author', 'figure', 
    'rights', 'reserved', 'permission', 'used', 'using', 'biorxiv', 'medrxiv', 'license', 'fig', 'fig.', 
    'al.', 'Elsevier', 'PMC', 'CZI'
]

for w in custom_stop_words:
    if w not in stopwords:
        stopwords.append(w)

In [ ]:
print(f"Rows Before: {len(data.index):20d}")
data.dropna(subset=['pdf_json_files'], inplace=True)
print(f"Rows Before: {len(data.index):20d}")

import json
import multiprocessing as mp

root_path = '/kaggle/input/CORD-19-research-challenge/'

def get_body_text(row):
    try:
        body_text = []
        
        with open(root_path + row.split('; ')[0], 'r') as f:
            content = json.load(f)
        body_text = []
        
        for entry in content['body_text']:
            body_text.append(entry['text'])

        body_text = '\n'.join(body_text)
        return body_text # Tokenizes the body text in one big go (hopefully faster!!)
    except:
        print('failed')
        return None

def get_numb_files(row): 
    return len(row.split('; '))

print(f"Avg Number of Files: {data['pdf_json_files'].progress_apply(get_numb_files).mean():14.2f}")

full_text = data['pdf_json_files'].progress_apply(get_body_text)
# full_text = data['abstract'] # Fuck naming convetions

data.to_csv('body_extract.csv', index=False)

print(f"Rows After: {len(data.index):10d}")

In [ ]:
import gc
gc.collect() # Does a little Garbage Collection

import scispacy

spacy.prefer_gpu(gpu_id=0) # Does not actually bottleneck as GPU performance is limited
parser = spacy.load("en_core_sci_lg")
parser.max_length = 7000000

def spacy_tokenizer(sentence):
    mytokens = parser(sentence)    
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]
    mytokens = [ word for word in mytokens if word not in stopwords and word not in punctuations ]
    mytokens = " ".join([i for i in mytokens])
    return mytokens

data['processed_text'] = full_text.progress_apply(spacy_tokenizer)
del full_text
del parser

In [ ]:
# Makes a histogram of paper length
from math import log10

log_function = log10

paper_length = data['processed_text'].progress_apply(lamda x: log_function(x + 1))

plt.hist(paper_length, bins=50)
plt.title("Log-Weighted Distribution of Paper Length")
plt.xlabel("Log Character Count")
plt.ylabel("Number of Papers")
plt.savefig("PaperLength.png")
plt.show()

del paper_length

In [ ]:
print('text after processing')
for i in range(3):
    print(f"        {i}. {data.loc['processed_text', i][:100]}")

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
def vectorize(text, maxx_features):
    
    vectorizer = TfidfVectorizer(max_features=maxx_features)
    X = vectorizer.fit_transform(text)
    return X

text = data['processed_text'].values
max_features = 2**12

X = vectorize(text, max_features)

import pickle
pickle.dump(X, open('X-first-pickle.p', 'wb'))

del test

### Mini EDA of Impact
Trying to linearize the distribution

I am going to take the log of this

In [ ]:
import matplotlib.pyplot as plt
import math

# Histograms
for plot_type in ['score', 'readers_count', 'posts', 'accounts']:
    print(plot_type)
    # Logs eveythings
    data[plot_type + "_linearized"] = data[plot_type].apply(lambda row: math.log10(row + 1))
    
    plt.hist(data[plot_type])
    plt.title(f"Impact Score of {plot_type}")
    plt.ylabel("# of Papers")
    plt.xlabel("Impact Score")
    plt.savefig(plot_type + ".png")
    plt.show()
    
    plt.hist(data[plot_type + "_linearized"])
    plt.title(f"Skew-Adjusted Impact Score of {plot_type}")
    plt.ylabel("# of Papers")
    plt.xlabel("Log10 of Impact Score")
    plt.savefig(plot_type + "_linearized.png")
    plt.show()
    
data['score'].max()

In [ ]:
# Score Histogram...

# Top 1 percentile analysis
top_1 = data['score'].quantile(q=.99, interpolation='linear')

top_1_percent_score = data['score'][data['score'] > top_1]

plt.hist(top_1_percent_score)
plt.ylabel("# of Papers")
plt.xlabel('Impact Score')
plt.title("Impact of Top 1% of Papers")
plt.savefig('top_1_score.png')
plt.show()

# Bottom 99 percentile analysis
bottom_99_percent_score = data['score'][data['score'] < top_1]

plt.hist(bottom_99_percent_score)
plt.ylabel("# of Papers")
plt.xlabel('Impact Score')
plt.title("Impact of Bottom 99% of Papers")
plt.savefig('bottom_99_score.png')
plt.show()

In [ ]:
# Linearizing the Score Data
from math import log10

top_1_lin = top_1_percent_score.apply(lambda x: log10(x + 1))

plt.hist(top_1_lin)
plt.title("Skew-Adjusted Impact of Top 1% of Papers")
plt.ylabel("# of Papers")
plt.xlabel("Log Impact")
plt.show()

bottom_99_lin = bottom_99_percent_score.apply(lambda x: log10(x + 1))

plt.hist(bottom_99_lin)
plt.title("Skew-Adjusted Impact of Bottom 99% of Papers")
plt.ylabel("# of Papers")
plt.xlabel("Log Impact")
plt.show()

# Clustering the Dataset 
1. Uses Principle Component Analysis (PCA) to turn 100 columns of data into 3 (enough to be plotted on a 3D dot plot).
2. Uses K-nearest neighbors to group papers based on the closest Euclidan distance
3. 2D Graph of the data
4. 3D graph of the data

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=0.8, random_state=42) # I would like this to be 95% ideally.
X_reduced = pca.fit_transform(X.toarray())
X.shape
X_reduced.shape

In [ ]:
# Prepares the Impact Data Using Linearization



In [ ]:
from sklearn.cluster import MiniBatchKMeans
from sklearn.cluster import KMeans

from sklearn import metrics
from scipy.spatial.distance import cdist

# run kmeans with many different k
distortions = []
K = [i for i in range(2, 30, 4)] # Does clusters in steps of 4 to save time...
for n_clusters in tqdm(K,total=len(K)):
    kmeans = KMeans(n_clusters=n_clusters, random_state=42).fit(X_reduced)
    # k_means.fit(X_reduced)
    distortions.append(sum(np.min(cdist(X_reduced, kmeans.cluster_centers_, 'euclidean'), axis=1)) / X.shape[0])

In [ ]:
X_line = [K[0], K[-1]]
Y_line = [distortions[0], distortions[-1]]

# Plot the elbow
plt.plot(K, distortions, 'b-')
plt.plot(X_line, Y_line, 'r')
plt.xlabel('k')
plt.ylabel('Distortion')
plt.title('The Elbow Method showing the optimal k')
plt.savefig('ElbowMethod.png')
plt.show()

In [ ]:
Y_fasdfs = X.copy()
print(X_reduced[:3,:3])
magnitudes = [np.linalg.norm(red) for red in X_reduced]
print(magnitudes[:3])
plt.hist(magnitudes[:min(len(magnitudes), 1000)], bins=30)
plt.savefig("reduced_weight.png")
plt.show()

In [ ]:
k = 18
kmeans = KMeans(n_clusters=k, random_state=42)
data['cluster'] = kmeans.fit_predict(X_reduced)
print(kmeans.score(X_reduced))

In [ ]:
from sklearn.manifold import TSNE

# TODO: turn up the perplexity for the actual run.

# The perplexity should be 50 for 200,000, but I am going to use 45 because I am dumb.,..
tsne = TSNE(verbose=1, perplexity=30, n_iter=1000)  # Changed perplexity from 100 to 50 per FAQ
X_embedded = tsne.fit_transform(X)

print(X.shape)

In [ ]:
import pickle

data.to_pickle('data_final.p')

pickle.dump(X, open('x.p', 'wb'))
pickle.dump(X_reduced, open('X_reduced.p', 'wb'))

#### Clustering the Top 1 % of Data

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns

# sns settings
sns.set(rc={'figure.figsize':(13,9)})

# colors
palette = sns.hls_palette(k, l=.4, s=.9)

# plot
sns.scatterplot(X_embedded[:,0], X_embedded[:,1], hue=data['cluster'], legend='full', palette=palette)
plt.title('t-SNE with Kmeans Labels')
plt.savefig("improved_cluster_tsne.png")
plt.show()

## Vectorization
Where the common vectors for each cluster are found

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
vectorizers = []
    
for ii in range(0, k):
    # Creating a vectorizer
    vectorizers.append(CountVectorizer(min_df=5, max_df=0.9, stop_words='english', lowercase=True, token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}'))

In [ ]:
vectorized_data = []

for current_cluster, cvec in enumerate(vectorizers):
    try:
        vectorized_data.append(cvec.fit_transform(data.loc[data['cluster'] == current_cluster, 'processed_text']))
    except Exception as e:
        print("Not enough instances in cluster: " + str(current_cluster))
        vectorized_data.append(None)

In [ ]:
# number of topics per cluster
NUM_TOPICS_PER_CLUSTER = 20


lda_models = []

for ii in range(0, k):
    # Latent Dirichlet Allocation Model
    lda = LatentDirichletAllocation(n_components=NUM_TOPICS_PER_CLUSTER, max_iter=10, learning_method='online',verbose=False, random_state=42)
    lda_models.append(lda)
    
lda_models[0]

In [ ]:
clusters_lda_data = []

for current_cluster, lda in enumerate(tqdm(lda_models, total=len(lda_models))):   
    if vectorized_data[current_cluster] != None:
        clusters_lda_data.append((lda.fit_transform(vectorized_data[current_cluster])))

In [ ]:
# Functions for printing keywords for each topic
def selected_topics(model, vectorizer, top_n=3):
    current_words = []
    keywords = []
    
    for idx, topic in enumerate(model.components_):
        words = [(vectorizer.get_feature_names()[i], topic[i]) for i in topic.argsort()[:-top_n - 1:-1]]
        for word in words:
            if word[0] not in current_words:
                keywords.append(word)
                current_words.append(word[0])
                
    keywords.sort(key = lambda x: x[1])  
    keywords.reverse()
    return_values = []
    for ii in keywords:
        return_values.append(ii[0])
    return return_values

In [ ]:
all_keywords = []
for current_vectorizer, lda in enumerate(tqdm(lda_models)):
    if vectorized_data[current_vectorizer] != None:
        all_keywords.append(selected_topics(lda, vectorizers[current_vectorizer]))

In [ ]:
f=open('topics.txt','w')

count = 0

for ii in all_keywords:

    if vectorized_data[count] != None:
        f.write(', '.join(ii) + "\n")
    else:
        f.write("Not enough instances to be determined. \n")
        f.write(', '.join(ii) + "\n")
    count += 1

f.close()

In [ ]:
print(all_keywords[:2][:5])

massive_all_keywords = []

for keywords in all_keywords:
    massive_all_keywords += keywords

keyword_count = {}
for keyword in massive_all_keywords:
    if keyword in keyword_count:
        keyword_count[keyword] += 1
    else:
        keyword_count[keyword] = 1
keyword_count = dict(sorted(keyword_count.items(), key=lambda item: item[1], reverse=True))

print(f"Number of Words in the Categories {len(massive_all_keywords):10d}")

n_bars = min(10, len(keyword_count.keys()))

plt.bar([i for i in range(n_bars)], list(keyword_count.values())[:n_bars], tick_label=list(keyword_count.keys())[:n_bars])
plt.xlabel("Word")
plt.ylabel("Number of Times Cited")
plt.title("Number of Times Each Work Was Cited")
plt.savefig("Category Commonalities")
plt.show()

# If greater than like 15, kill them. Also, maybe categoies should be shrunk to like 15, a lot more readale this way



### Plotting (Where things become nice and pretty!)

In [ ]:
! wget https://raw.githubusercontent.com/MaksimEkin/COVID19-Literature-Clustering/master/lib/plot_text.py
! wget https://raw.githubusercontent.com/MaksimEkin/COVID19-Literature-Clustering/master/lib/call_backs.py
! mkdir library
! mv plot_text.py library/.
! mv call_backs.py library/.
! touch library/__init__.py
! ls library/

In [ ]:
# required libraries for plot
from library.plot_text import header, description, description2, cite, description_search, description_slider, notes, dataset_description, toolbox_header 
from library.call_backs import input_callback, selected_code
import bokeh
from bokeh.models import ColumnDataSource, HoverTool, LinearColorMapper, CustomJS, Slider, TapTool, TextInput
from bokeh.palettes import Category20
from bokeh.transform import linear_cmap, transform
from bokeh.io import output_file, show, output_notebook
from bokeh.plotting import figure
from bokeh.models import RadioButtonGroup, TextInput, Div, Paragraph
from bokeh.layouts import column, widgetbox, row, layout
from bokeh.layouts import column

In [ ]:
import os

topic_path = os.path.join(os.getcwd(), 'topics.txt')
with open(topic_path) as f:
    topics = f.readlines()

In [ ]:
# show on notebook
output_notebook()
# target labels
y_labels = data['cluster']

# data sources
source = ColumnDataSource(data=dict(
    x= X_embedded[:,0], 
    y= X_embedded[:,1],
    x_backup = X_embedded[:,0],
    y_backup = X_embedded[:,1],
    desc= y_labels, 
    titles= data['title'],
    authors = data['authors'],
    journal = data['journal'],
    abstract = data['abstract'],
    labels = ["C-" + str(x) for x in y_labels],
    links = data['doi'],
    impact = data['score'].apply(lambda r: round(r, 2))
    ))

# hover over information
hover = HoverTool(tooltips=[
    ("Title", "@titles{safe}"),
    ("Author(s)", "@authors{safe}"),
    ("Journal", "@journal"),
    ("Abstract", "@abstract{safe}"),
    ("Link", "@links")
    ("Impact", "@impact")
],
point_policy="follow_mouse")

# map colors
mapper = linear_cmap(field_name='desc', 
                     palette=Category20[20],
                     low=min(y_labels) ,high=max(y_labels))

# prepare the figure
plot = figure(plot_width=1200, plot_height=850, 
           tools=[hover, 'pan', 'wheel_zoom', 'box_zoom', 'reset', 'save', 'tap'], 
           title="Clustering of the COVID-19 Literature with t-SNE and K-Means", 
           toolbar_location="above")

# plot settings
plot.scatter('x', 'y', size=5, 
          source=source,
          fill_color=mapper,
          line_alpha=0.3,
          line_color="black",
          legend = 'labels')
plot.legend.background_fill_alpha = 0.6

In [ ]:
# Keywords
text_banner = Paragraph(text= 'Keywords: Slide to specific cluster to see the keywords.', height=25)
input_callback_1 = input_callback(plot, source, text_banner, topics)

# currently selected article
div_curr = Div(text="""Click on a plot to see the link to the article.""",height=150)
callback_selected = CustomJS(args=dict(source=source, current_selection=div_curr), code=selected_code())
taptool = plot.select(type=TapTool)
taptool.callback = callback_selected

# WIDGETS
slider = Slider(start=0, end=20, value=20, step=1, title="Cluster #")
keyword = TextInput(title="Search:", callback=input_callback_1)

# pass call back arguments
input_callback_1.args["text"] = keyword
input_callback_1.args["slider"] = slider
# column(,,widgetbox(keyword),,widgetbox(slider),, notes, cite, cite2, cite3), plot

In [ ]:
# STYLE
header.sizing_mode = "stretch_width"
header.style={'color': '#2e484c', 'font-family': 'Julius Sans One, sans-serif;'}
header.margin=5

description.style ={'font-family': 'Helvetica Neue, Helvetica, Arial, sans-serif;', 'font-size': '1.1em'}
description.sizing_mode = "stretch_width"
description.margin = 5

description2.sizing_mode = "stretch_width"
description2.style ={'font-family': 'Helvetica Neue, Helvetica, Arial, sans-serif;', 'font-size': '1.1em'}
description2.margin=10

description_slider.style ={'font-family': 'Helvetica Neue, Helvetica, Arial, sans-serif;', 'font-size': '1.1em'}
description_slider.sizing_mode = "stretch_width"

description_search.style ={'font-family': 'Helvetica Neue, Helvetica, Arial, sans-serif;', 'font-size': '1.1em'}
description_search.sizing_mode = "stretch_width"
description_search.margin = 5

slider.sizing_mode = "stretch_width"
slider.margin=15

keyword.sizing_mode = "scale_both"
keyword.margin=15

div_curr.style={'color': '#BF0A30', 'font-family': 'Helvetica Neue, Helvetica, Arial, sans-serif;', 'font-size': '1.1em'}
div_curr.sizing_mode = "scale_both"
div_curr.margin = 20

text_banner.style={'color': '#0269A4', 'font-family': 'Helvetica Neue, Helvetica, Arial, sans-serif;', 'font-size': '1.1em'}
text_banner.sizing_mode = "stretch_width"
text_banner.margin = 20

plot.sizing_mode = "scale_both"
plot.margin = 5

dataset_description.sizing_mode = "stretch_width"
dataset_description.style ={'font-family': 'Helvetica Neue, Helvetica, Arial, sans-serif;', 'font-size': '1.1em'}
dataset_description.margin=10

notes.sizing_mode = "stretch_width"
notes.style ={'font-family': 'Helvetica Neue, Helvetica, Arial, sans-serif;', 'font-size': '1.1em'}
notes.margin=10

cite.sizing_mode = "stretch_width"
cite.style ={'font-family': 'Helvetica Neue, Helvetica, Arial, sans-serif;', 'font-size': '1.1em'}
cite.margin=10

r = row(div_curr,text_banner)
r.sizing_mode = "stretch_width"

In [ ]:
# LAYOUT OF THE PAGE
l = layout([
    [header],
    [description],
    [description_slider, description_search],
    [slider, keyword],
    [text_banner],
    [div_curr],
    [plot],
    [description2, dataset_description, notes, cite],
])
l.sizing_mode = "scale_both"


# show
output_file('t-sne_covid-19_interactive.html')
show(l)

#### Clustering the Bottom 99 Percent of Data

## Conclusion
    All in all, it was found that there was high correlation between each clusters, as well as a good corelation between impact and cluster size.